In [120]:
import os
from pytubefix import YouTube
from pytubefix.cli import on_progress

In [121]:
url = "https://www.youtube.com/shorts/Ta6HEAFXK3k"

yt = YouTube(url, on_progress_callback = on_progress)
print(yt.title)
 
ys = yt.streams.get_highest_resolution()
ys.download()

조림왕 최강록


'c:\\Users\\dgh00\\OneDrive\\바탕 화면\\whisper\\조림왕 최강록.mp4'

In [122]:
print(f"title: {yt.title}")
print(f"views : {yt.views}")
print(f"description: {yt.description}")
print(f"length: {yt.length}")

title: 조림왕 최강록
views : 6030400
description: #shorts #최강록 #강레오 #마스터셰프코리아
length: 50


In [123]:
# rename the mp4 to the title of the video without a file extension
os.rename(yt.title + ".mp4", yt.title)

In [124]:
# extract audio from the video
import time
import math
import ffmpeg

def extract_audio(input_file):
    extracted_audio = f"audio-{input_file}.wav"
    stream = ffmpeg.input(input_file)
    stream = ffmpeg.output(stream, extracted_audio)
    ffmpeg.run(stream, overwrite_output=True)
    return extracted_audio

In [125]:
audio_extract = extract_audio(yt.title)

In [126]:
# transcribe the audio
from faster_whisper import WhisperModel
def transcribe(audio):

    model = WhisperModel("medium", device="cpu")
    segments, info = model.transcribe(audio)
    language = info[0]
    print(f"transcription language: {language}")
    segments = list(segments) # this is where the transcribe happens

    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

    return language, segments

In [127]:
%%time

language, segments = transcribe(audio_extract)

transcription language: ko
[0.00s -> 1.88s]  최강록 씨 앞으로 나오세요.
[2.24s -> 8.00s]  보통 이제 항정살을 많이 구워서 드시는 것 같은데요.
[8.08s -> 11.76s]  졸여도 부드럽게 드실 수 있을 것 같아서요.
[11.76s -> 16.48s]  한 가지는 간장으로 졸여보고 하나는 쌈장만 넣어서 했으면 좋겠습니다.
[21.28s -> 24.68s]  강래호 심사위원님이 쏘는 눈빛.
[25.16s -> 27.84s]  거기 마주치면 한 6개월씩 늘어요.
[28.80s -> 30.36s]  진짜 늙어요.
[30.36s -> 36.84s]  최강록 씨 오늘 최강록 씨한테 나는 재킬라고 하이드를 본 기분이에요.
[36.92s -> 39.44s]  그 아이디어나 맛을 제일 좋은 걸로 가봤어요.
[40.24s -> 46.74s]  미소 매칭 먹은 모시라콘은 절정적인 맛이었어요.
[46.74s -> 49.24s]  정말 맛있는 항정살 조림이었습니다.
[49.24s -> 50.24s]  수고하셨습니다.
CPU times: total: 2min 56s
Wall time: 48.4 s


In [128]:
for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

[0.00s -> 1.88s]  최강록 씨 앞으로 나오세요.
[2.24s -> 8.00s]  보통 이제 항정살을 많이 구워서 드시는 것 같은데요.
[8.08s -> 11.76s]  졸여도 부드럽게 드실 수 있을 것 같아서요.
[11.76s -> 16.48s]  한 가지는 간장으로 졸여보고 하나는 쌈장만 넣어서 했으면 좋겠습니다.
[21.28s -> 24.68s]  강래호 심사위원님이 쏘는 눈빛.
[25.16s -> 27.84s]  거기 마주치면 한 6개월씩 늘어요.
[28.80s -> 30.36s]  진짜 늙어요.
[30.36s -> 36.84s]  최강록 씨 오늘 최강록 씨한테 나는 재킬라고 하이드를 본 기분이에요.
[36.92s -> 39.44s]  그 아이디어나 맛을 제일 좋은 걸로 가봤어요.
[40.24s -> 46.74s]  미소 매칭 먹은 모시라콘은 절정적인 맛이었어요.
[46.74s -> 49.24s]  정말 맛있는 항정살 조림이었습니다.
[49.24s -> 50.24s]  수고하셨습니다.


In [129]:
# SubRip(SRT)
# Subtitle Index: 0, 1, 2
# Timecode: Start and end markers. HH:MM:SS,sss format.
# Text: The subtitle text.

In [130]:
# this is a helper function that takes time in seconds and
# converts it into HH:MM:SS,sss format for the SRT subtitle files.

def format_time_for_srt(seconds):
    hours = math.floor(seconds / 3600)
    seconds %= 3600
    minutes = math.floor(seconds / 60)
    seconds %= 60
    miliseconds = round((seconds - math.floor(seconds)) * 1000)
    seconds=math.floor(seconds)
    formatted_time = f"{hours :02d}:{minutes :02d}:{seconds :02d},{miliseconds :03d}"

    return formatted_time

In [131]:
def generate_subtitle_file(input_file, language, segments):
    subtitle_file = f"sub-{input_file}.{language}.srt"
    text = ""
    for index, segment in enumerate(segments):
        segment_start = format_time_for_srt(segment.start)
        segment_end = format_time_for_srt(segment.end)

        text += f"{str(index + 1)}\n"
        text += f"{segment_start} --> {segment_end}\n"
        text += f"{segment.text}\n\n"

    # UTF-8 인코딩으로 파일을 열기
    with open(subtitle_file, 'w', encoding='utf-8') as f:
        f.write(text)

    return subtitle_file

In [132]:
subtitle_file = generate_subtitle_file(yt.title, language, segments)

In [133]:
def add_subtitle_to_video(input_file, subtitle_file, subtitle_language):
    video_input_stream = ffmpeg.input(input_file)
    subtitle_input_stream = ffmpeg.input(subtitle_file)
    output_video = f"output-{input_file}-{subtitle_language}.mp4"
    subtitle_track_title = subtitle_file.replace(".srt", "")
    stream = ffmpeg.output(video_input_stream, output_video,
                           vf = f"subtitles='{subtitle_file}'")
    ffmpeg.run(stream, overwrite_output=True)

In [134]:
add_subtitle_to_video(yt.title, subtitle_file, language)